# Serialization, loading and saving of zfit objects

The long-term goal is to be able to save and load zfit objects, such as models, spaces, parameters, etc. This is not yet fully implemented, but some parts are already available, some stable, some more experimental.

Overview:
- Binary (pickle) loading and dumping of (frozen) `FitResult` is fully available
- Human-readable serialization (also summarized under HS3) of
 - parameters and models is available, but not yet stable
 - losses and datasets are not yet available


In [ ]:
import pickle

import mplhep
import numpy as np
import zfit
import zfit.z.numpy as znp
from matplotlib import pyplot as plt
from zfit import z

In [ ]:
mu = zfit.Parameter("mu", 1.2, -4, 5)
sigma = zfit.Parameter("sigma", 3, 0, 10)
obs = zfit.Space("obs1", limits=(-10, 20))
model = zfit.pdf.Gauss(mu=mu, sigma=sigma, obs=obs)

In [ ]:
data = model.sample(10000)
loss = zfit.loss.UnbinnedNLL(model=model, data=data)

In [ ]:
minimizer = zfit.minimize.Minuit()

In [ ]:
x = np.linspace(*obs.limit1d, 1000)
mu.set_value(1.5)
sigma.set_value(2)
mplhep.histplot(data.to_binned(50), density=True, label="data")
plt.plot(x, model.pdf(x), label="model pre fit")
result = minimizer.minimize(loss)
plt.plot(x, model.pdf(x), label="model post fit")
plt.legend()

In [ ]:
result.freeze()
dumped_result = pickle.dumps(result)

In [ ]:
loaded_result = pickle.loads(dumped_result)

In [ ]:
mu.set_value(0.42)
print(f"mu before: {mu.value()}")
zfit.param.set_values(params=model.get_params(), values=loaded_result)
print(f"mu after: {mu.value()}, set to result value: {loaded_result.params[mu]['value']}")

In [ ]:
model.to_dict()

In [ ]:
gauss2 = model.from_dict(model.to_dict())  # effectively creates a copy (parameters are shared!)

In [ ]:
mu.to_dict()

In [ ]:
zfit.hs3.dump(model)

In [ ]:
zfit.hs3.load(zfit.hs3.dump(model))

In [ ]:
hs3obj = zfit.hs3.load(zfit.hs3.dump(model))

In [ ]:
list(hs3obj['pdfs'].values())

In [ ]:
zfit.hs3.dump(hs3obj)

In [ ]:
'pdfs' in hs3obj